We have seen the steps regarding chains in previous notebooks. We determined the steps to be taken in these chains. But what happens if language models determine the steps to be taken? We will look for the answer to this question in this notebook. Here, we will enable language models to make their own decisions by using agents.

Agents use language models to interact with other agents. In other words, agents choose the action to be taken themselves. We will use two vehicles. One of them is retrievers and the other tool is Search Engine. Search engine answers questions that require up-to-date information.


In [1]:
from dotenv import load_dotenv

load_dotenv() 

True

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://blog.langchain.dev/langgraph/") # WebBaseLoader is a document loader that uses the web to load documents

docs = loader.load() # Load the documents from the web
print(docs)

[Document(page_content='\n\n\nLangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nCase Studies\n\n\n\n\nLangChain\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\nBy LangChain\n7 min read\nJan 17, 2024\n\n\n\n\n\nTL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we wi

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter() # RecursiveCharacterTextSplitter is a text splitter that splits the text into characters recursively

documents = text_splitter.split_documents(docs) # Split the documents into characters. Documents are a list of strings
print(documents)

[Document(page_content='LangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nCase Studies\n\n\n\n\nLangChain\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\nBy LangChain\n7 min read\nJan 17, 2024', metadata={'source': 'https://blog.langchain.dev/langgraph/', 'title': 'LangGraph', 'language': 'en'}), Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an

In [4]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings() # To convert the text to vectors, we use OpenAIEmbeddings

In [5]:
from langchain_community.vectorstores import FAISS

vector = FAISS.from_documents(documents, embeddings) # The data was first converted into numerical representations with embeddings and then indexed into the database. Vectors were made quickly searchable using FAISS.

In [6]:
retriever = vector.as_retriever() # Retriever has data about langgraph in db.
# We create a retriever to capture this indexed data.

In [7]:
retriever.get_relevant_documents("What is LangGraph")[0] # We can get the relevant documents with the retriever.

C:\Users\yasin\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we will first walk through the motivations for LangGraph. We will then cover the basic functionality it provides. We will then spotlight two agent runtimes we\'ve implemented already. We will then highlight a few of the common modifications to these runtimes we\'ve heard requests for, and examples of implementing those. We will finish with a preview of what we will be releasing next.MotivationOne of the big va

In [8]:
from langchain.tools.retriever import create_retriever_tool # We created a tool to capture the information we want

retriever_tool = create_retriever_tool(
    retriever,
    "langgraph_search", # Tool name.  This will be passed to the language model, so should be unique and somewhat descriptive.
    "Search for information about LangGraph. For any question about LangGraph, you must use this tool!", # The description for the tool. This will be passed to the language model, so should be descriptive.
) # We create a retriever tool to get the relevant documents. It return tool class to pass to an agent. 

Now we will use tavily as the Search tool. Tavily was developed specifically for artificial intelligence agents. 

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults # Tool that queries the Tavily Search API and gets back json.
# tavily is a search engine tool that uses the Tavily Search API to search for information.

search = TavilySearchResults() # We created a tool to search.

In [10]:
search.invoke("Which is the best language model?") # We can get the search results with the search tool.

[{'url': 'https://www.techtarget.com/WhatIs/feature/12-of-the-best-large-language-models',
  'content': 'Process mining is a technique that interprets logs from enterprise applications -- like customer resource management (CRM) and ...\nOrganizational change management (OCM) is a type of change management framework for managing the effect of new business processes...\nAn employee resource group is a workplace club or more formally realized affinity group organized around a shared interest or ...\nEmployee training and development is a set of activities and programs designed to enhance the knowledge, skills and abilities of ...\nEmployee sentiment analysis is the use of natural language processing and other AI techniques to automatically analyze employee ...\nCustomer profiling is the detailed and systematic process of constructing a clear portrait of a company\'s ideal customer by ...\nCustomer insight, also known as consumer insight, is the understanding and interpretation of customer

In [11]:
tools = [retriever_tool, search] # We have two tools. One of them is a retriever and the other is a search tool.

The tools to be used have been set, now we will create the agent. The agent will use the tools to interact with other agents.

In [12]:
# We created a model to guide the agent
from langchain_openai import ChatOpenAI # ChatOpenAI is a class that uses OpenAI to interact with the language model 

model = ChatOpenAI(
    temperature=0
) # We created a model to interact with the language model. 

In [17]:
# now we will write a prompt to direct the agent. We will use a ready-made prompt template.
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent") # We pulled the prompt template from the hub. https://smith.langchain.com/hub/hwchase17/openai-functions-agent

In [18]:
prompt.messages # We can see the messages in the prompt.

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Now we will initialize an agent using model, tool and prompt.

The prompt and agent are responsible for receiving input and deciding what actions to take.

In [20]:
from langchain.agents import create_openai_functions_agent # Create an agent that uses OpenAI function calling.

agent = create_openai_functions_agent(
    model,
    tools,
    prompt
) # We created an agent to interact with the language model.
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, 

In [21]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools,
    verbose=True, # If True, print the agent's messages.
) # We created an agent executor to execute the agent.

In [22]:
agent_executor.invoke(
    {
        "input": "Hi!?", # The input message to the agent.
    }
) # We executed the agent.



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'Hi!?', 'output': 'Hello! How can I assist you today?'}

In [23]:
agent_executor.invoke(
    {
        "input": "What is the weather in Istanbul?", # The input message to the agent.
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Istanbul'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Istanbul', 'region': 'Istanbul', 'country': 'Turkey', 'lat': 41.02, 'lon': 28.96, 'tz_id': 'Europe/Istanbul', 'localtime_epoch': 1716192750, 'localtime': '2024-05-20 11:12'}, 'current': {'last_updated_epoch': 1716192000, 'last_updated': '2024-05-20 11:00', 'temp_c': 19.0, 'temp_f': 66.2, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.9, 'wind_kph': 19.1, 'wind_degree': 20, 'wind_dir': 'NNE', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 64, 'cloud': 50, 'feelslike_c': 19.0, 'feelslike_f': 66.2, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 6.0, 'gust_mph': 16.3, 'gust_kph': 26.3}}"}, {'url': 'https://www.accuweather.com/en/tr/istanbul/318251/current-weather/318

{'input': 'What is the weather in Istanbul?',
 'output': 'The current weather in Istanbul is partly cloudy with a temperature of 19.0°C (66.2°F). The wind speed is 19.1 km/h coming from the north-northeast direction. The humidity is at 64%, and the visibility is 10.0 km.'}

To provide weather information, the agent gave us information using the tavily search engine. He provided us with information by obtaining information from certain websites.

We used two tools. Now we will ask the agent about the langgraph and this time we will use the retriever tool. Agent will use the retriever tool to get the information.

In [24]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Can I use LangGraph for agent runtimes?"), # We asked the agent about the langgraph.
    AIMessage(content="Yes"),
] # We created a chat history to interact with the agent.

Now we will interact with the agent using the chat history we created.

In [25]:
agent_executor.invoke(
    {
        "chat_history": chat_history,
        "input": "Tell me how", # The input message to the agent.
    }
) # We executed the agent.



> Entering new AgentExecutor chain...

Invoking: `langgraph_search` with `{'query': 'using LangGraph for agent runtimes'}`


TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we will first walk through the motivations for LangGraph. We will then cover the basic functionality it provides. We will then spotlight two agent runtimes we've implemented already. We will then highlight a few of the common modifications to these runtimes we've heard requests for, and examples of implement

{'chat_history': [HumanMessage(content='Can I use LangGraph for agent runtimes?'),
  AIMessage(content='Yes')],
 'input': 'Tell me how',
 'output': 'LangGraph is a module built on top of LangChain that enables the creation of cyclical graphs, which are often needed for agent runtimes. LangGraph provides functionality through the StateGraph class, which represents the graph and allows for the addition of nodes and edges. Nodes in the graph update a central state object over time, and edges connect the nodes to create the flow of the graph.\n\nTo use LangGraph for agent runtimes, you can follow these steps:\n1. Initialize a StateGraph by passing in a state definition.\n2. Add nodes to the graph using the `graph.add_node(name, value)` syntax, where the name is a string to refer to the node and the value is a function or LCEL runnable.\n3. Add edges to connect the nodes in the graph, including starting edges, normal edges, and conditional edges.\n4. Compile the graph into a runnable using 

The agent fulfilled the tasks I gave him. He made the decision himself while performing these duties. Tavily and retriever evaluated the tools according to the input given and presented us with the results accordingly.